In [3]:
import openai
import getpass
import os

#openai.api_key = getpass.getpass("Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = getpass.getpass()


 ········


In [18]:
import requests
from PIL import Image
# Download the FDA-provided sample image
image_url = "https://www.fda.gov/media/137352/download"  # Replace with another link if necessary
image_path = "nutrition_label_sample.png"

# Download and convert the image
response = requests.get(image_url)
with open("temp_image.pdf", "wb") as f:
    f.write(response.content)

In [7]:
import os

# Check file size to ensure it downloaded correctly
if os.path.exists("temp_image.pdf"):
    file_size = os.path.getsize("temp_image.pdf")
    print(f"File downloaded successfully, size: {file_size} bytes.")
else:
    print("File download failed. Please check the URL.")


File downloaded successfully, size: 155847 bytes.


In [8]:
with open("temp_image.pdf", "rb") as f:
    content = f.read(100)  # Read the first 100 bytes
    print(content)


b'%PDF-1.7\r%\xe2\xe3\xcf\xd3\r\n21 0 obj\r<</Linearized 1/L 150436/O 24/E 145152/N 1/T 149901/H [ 1176 278]>>\rendobj\r'


In [9]:
pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 32.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import fitz  # PyMuPDF

try:
    pdf_document = fitz.open("temp_image.pdf")
    page = pdf_document[0]  # Get the first page
    pix = page.get_pixmap()
    pix.save("nutrition_label_sample.png")  # Save as PNG
    print("PDF successfully converted to PNG.")
except Exception as e:
    print(f"Error during PDF conversion: {e}")


PDF successfully converted to PNG.


In [11]:
if os.path.exists("nutrition_label_sample.png"):
    file_size = os.path.getsize("nutrition_label_sample.png")
    print(f"Image file created successfully, size: {file_size} bytes.")
else:
    print("Image file creation failed.")


Image file created successfully, size: 38137 bytes.


In [12]:
try:
    base64_image = encode_image("nutrition_label_sample.png")
    print("Image successfully encoded to base64.")
except Exception as e:
    print(f"Error during base64 encoding: {e}")


Image successfully encoded to base64.


In [17]:
import base64
from openai import OpenAI

# Initialize the OpenAI client
client = OpenAI()

# Function to encode the image as base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Path to the PNG file created in earlier steps
image_path = "nutrition_label_sample.png"

# Encode the PNG file in base64
try:
    base64_image = encode_image(image_path)
    print("Image successfully encoded to base64.")
except Exception as e:
    print(f"Error during base64 encoding: {e}")
    raise e

# Call the OpenAI Vision API with the base64-encoded image
try:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Extract the nutritional information from this label:"},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}},
                ],
            }
        ],
        max_tokens=300,
    )

    # Display the response
    print("Extracted Nutritional Information:")
    print(response.choices[0].message.content)
except Exception as e:
    print(f"Error during API call: {e}")
    raise e


Image successfully encoded to base64.
Extracted Nutritional Information:
Here’s the nutritional information extracted from the label:

- **Serving Size:** 2/3 cup (55g)
- **Calories per serving:** 230

### Nutritional Information

- **Total Fat:** 8g (10% Daily Value)
  - Saturated Fat: 1g (5% Daily Value)
  - Trans Fat: 0g
- **Cholesterol:** 0mg (0% Daily Value)
- **Sodium:** 160mg (7% Daily Value)
- **Total Carbohydrate:** 37g (13% Daily Value)
  - Dietary Fiber: 4g (14% Daily Value)
  - Total Sugars: 12g 
    - Includes 10g Added Sugars (20% Daily Value)
- **Protein:** 3g

### Vitamins and Minerals
- **Vitamin D:** 2mcg (10% Daily Value)
- **Calcium:** 260mg (20% Daily Value)
- **Iron:** 8mg (45% Daily Value)
- **Potassium:** 240mg (6% Daily Value) 

**Note:** % Daily Values are based on a 2,000 calorie diet.
